In [1]:
import numpy as np
import pandas as pd
from decoding import get_mean_dff_traces, run_decoding
from allen_v1dd.client import OPhysClient, OPhysSession
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [2]:
## Path to locally saved NWB files
base_folder = "/home/naomi/Desktop/data/V1dd_nwbs"
client = OPhysClient(base_folder)
print(client.get_all_session_ids())

## Load in metrics
metrics_df = pd.read_csv("/home/naomi/Desktop/data/all_metrics_240426.csv")
metrics_df.head()

['M409828_11', 'M409828_12', 'M409828_13', 'M409828_14', 'M409828_15', 'M409828_16', 'M409828_17', 'M409828_18', 'M409828_19', 'M409828_1a', 'M409828_1b', 'M409828_1c', 'M409828_1d', 'M409828_1e', 'M409828_1f', 'M409828_21', 'M409828_22', 'M409828_23', 'M409828_24', 'M409828_25', 'M409828_31', 'M409828_32', 'M409828_33', 'M409828_34', 'M409828_35', 'M409828_41', 'M409828_42', 'M409828_43', 'M409828_44', 'M409828_45', 'M409828_51', 'M409828_52', 'M409828_53', 'M409828_54', 'M409828_55', 'M416296_11', 'M416296_12', 'M416296_13', 'M416296_14', 'M416296_15', 'M416296_17', 'M416296_18', 'M416296_19', 'M416296_1a', 'M416296_1b', 'M416296_1c', 'M416296_21', 'M416296_22', 'M416296_23', 'M416296_24', 'M416296_25', 'M416296_31', 'M416296_32', 'M416296_33', 'M416296_34', 'M416296_35', 'M416296_41', 'M416296_42', 'M416296_43', 'M416296_44', 'M416296_45', 'M416296_51', 'M416296_52', 'M416296_53', 'M416296_54', 'M416296_55', 'M427836_11', 'M427836_12', 'M427836_13', 'M427836_14', 'M427836_15', 'M427

,Unnamed: 0,orig_cell_index,mouse_id,valid_cell_index,cell_index,col_vol,column,volume,plane,x,...,dgf_pref_dir,dgw_pref_dir,dgf_osi,dgw_osi,dgf_dsi,dgw_dsi,dgf_ls,dgw_ls,ns12_ls,ns118_ls
0,0,8542,slc2,0,0,16,1,6,0,236,...,300,150,0.214924,0.485850,0.143261,0.394904,0.650162,0.670042,0.941426,0.943378
1,1,8543,slc2,1,1,16,1,6,0,159,...,300,90,0.765341,0.699801,0.934434,0.180604,0.914330,0.874348,0.962018,0.940841
2,2,8544,slc2,2,2,16,1,6,0,193,...,300,240,0.447789,0.317350,0.389030,0.267353,0.655039,0.657950,0.954018,0.956597
3,3,8545,slc2,3,3,16,1,6,0,230,...,210,0,0.661951,0.464806,0.056502,0.402281,0.667079,0.674886,0.944030,0.944862
4,4,8546,slc2,4,4,16,1,6,0,61,...,330,330,0.503441,0.748724,0.523120,1.000000,0.684561,0.845968,0.962426,0.954630


In [3]:
## Select one session
session_id = 'M409828_1f'
sess = client.load_ophys_session(session_id)
print(f"Loaded ophys session {session_id}, column {sess.get_column_id()}, volume {sess.get_volume_id()}, depth {sess.get_plane_depth(plane=1)}")

Loaded ophys session M409828_1f, column 1, volume f, depth 725


In [4]:
get_mean_dff_traces(session=sess, stimulus_type="drifting_gratings_full", plane=1)

,0,1,2,3,4,5,6,7,8,9,...,32,34,38,44,59,67,69,70,74,90
0,0.005685,0.122026,-0.066326,-0.034709,0.032532,0.106559,0.031938,0.046149,0.038758,0.002242,...,-0.031829,-0.014306,0.064121,-0.009470,-0.013960,-0.029749,0.039002,2.171099,0.016498,0.030788
1,-0.005558,-0.025211,0.060530,0.072690,-0.019783,0.036175,0.046481,0.038596,0.022011,-0.032739,...,0.057949,0.034134,0.069019,0.046537,0.002091,0.040191,0.045730,1.775979,0.755839,-0.018843
2,-0.013601,0.081380,0.014984,-0.016622,0.023069,0.021365,0.014716,0.045413,-0.040319,0.002124,...,0.024548,-0.051056,0.527822,0.006894,0.001932,0.003908,0.114763,0.542036,-0.031065,-0.063689
3,0.031529,0.032925,0.095078,-0.019028,0.036305,-0.039034,0.004986,-0.024235,-0.020224,-0.030264,...,-0.029229,0.020890,-0.050396,0.396566,0.048758,0.006598,0.406250,-0.044001,0.011618,0.078684
4,0.070829,0.084478,0.066810,0.082585,0.014641,0.018382,0.017346,0.007917,0.025679,-0.025525,...,-0.009963,-0.009457,0.055557,0.012667,0.025116,-0.008148,-0.010143,0.068914,-0.014628,0.079100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0.013626,-0.017717,0.045549,-0.026348,0.013150,0.001370,-0.015400,-0.012343,0.028818,-0.020287,...,0.060941,0.060282,-0.028079,-0.004698,0.007778,0.053282,0.005442,0.035739,0.026524,0.100435
182,0.020433,0.064589,0.006423,-0.026870,0.031079,-0.028506,0.031044,0.004047,0.011950,0.017903,...,-0.009108,0.032963,0.007189,0.001061,0.062201,0.032796,-0.002764,0.023962,-0.006495,0.075312
183,0.085694,0.031030,-0.052685,0.060693,0.009708,-0.018306,0.006002,0.073240,-0.019688,0.008652,...,0.056351,0.019542,0.093070,-0.053112,-0.011393,0.121200,0.037603,0.116303,-0.039537,-0.042055
184,0.021324,0.100352,0.010178,-0.007876,-0.080852,0.000960,0.009300,0.025488,0.024116,0.018056,...,0.040372,0.033398,0.087489,0.041825,-0.007790,0.041849,-0.009605,0.021645,-0.026890,-0.035924


In [5]:
run_decoding(
    session=sess,
    plane=1,
    stimulus_type="drifting_gratings_full",
    repetitions=5,
    decode_dim="direction",
    max_neighbors=15,
    metric="correlation",
    # test_size=0.2,
    bootstrap=True,
    bootstrap_size=30,
    metrics_df=metrics_df,
    unduplicated=False,
    folder_name="/home/naomi/Desktop/data",
    save_decoding=False,
    results_folder="/home/naomi/Desktop/data/decoding_results",
    tag="250720"
)

,mouse_id,column_id,volume_id,plane,repetition_num,test_accuracies,test_accuracies_mean,test_accuracies_std,shuf_test_accuracies,shuf_test_accuracies_mean,shuf_test_accuracies_std,num_k_neighbors,shuf_num_k_neighbors
0,409828,1,f,1,0,"[0.21052631578947367, 0.13513513513513514, 0.2...",0.193457,0.049095,"[0.15789473684210525, 0.08108108108108109, 0.1...",0.128876,0.042633,15,6
1,409828,1,f,1,1,"[0.18421052631578946, 0.2702702702702703, 0.08...",0.155761,0.066626,"[0.07894736842105263, 0.05405405405405406, 0.1...",0.086060,0.031603,7,13
2,409828,1,f,1,2,"[0.13157894736842105, 0.13513513513513514, 0.1...",0.145235,0.055808,"[0.15789473684210525, 0.10810810810810811, 0.1...",0.134282,0.037652,11,7
3,409828,1,f,1,3,"[0.13157894736842105, 0.16216216216216217, 0.0...",0.112802,0.049251,"[0.18421052631578946, 0.13513513513513514, 0.1...",0.128734,0.034222,8,9
4,409828,1,f,1,4,"[0.15789473684210525, 0.1891891891891892, 0.13...",0.161309,0.038260,"[0.13157894736842105, 0.13513513513513514, 0.0...",0.112802,0.052132,13,7


In [6]:
from decoding import *

In [7]:
len(select_rois(session=sess, plane=1, metrics_df=metrics_df, unduplicated=False))

33

In [8]:
sess_id = 'M409828_1f'
sess = client.load_ophys_session(sess_id)

X_data = get_X_data(
    session=sess,
    plane=1,
    stimulus_type="drifting_gratings_full",
    metrics_df=metrics_df,
    unduplicated=True,
    folder_name="/home/naomi/Desktop/data",
)

X_data

array([[ 0.00568497,  0.12202562, -0.0663263 , ...,  2.171099  ,
         0.01649844,  0.03078773],
       [-0.00555836, -0.02521069,  0.06053013, ...,  1.7759787 ,
         0.75583917, -0.01884298],
       [-0.01360072,  0.08137986,  0.01498445, ...,  0.54203635,
        -0.03106521, -0.06368918],
       ...,
       [ 0.08569397,  0.03102981, -0.05268541, ...,  0.11630315,
        -0.03953739, -0.04205528],
       [ 0.0213242 ,  0.10035192,  0.01017784, ...,  0.02164455,
        -0.02689013, -0.03592373],
       [ 0.01930217,  0.05575607, -0.02660632, ...,  0.03850644,
         0.04099073,  0.05771739]], dtype=float32)

In [9]:
stim_types = [
            "drifting_gratings_full", 
            "drifting_gratings_windowed", 
            "natural_images_12", 
            "natural_images"
            ]
decode_dims = {
    "drifting_gratings_full": ["direction", "spatial"],
    "drifting_gratings_windowed": ["direction", "spatial"],
    "natural_images_12": ["image_index"],
    "natural_images": ["image_index"]}

In [ ]:
for sess_id in client.get_all_session_ids():
# for sess_id in ['M409828_21']:
    try:
        sess = client.load_ophys_session(sess_id)
    except ValueError as e:
        print(f"Error loading session {sess_id}: {e}")
        continue
    column = sess.get_column_id()
    num_planes = len(sess.get_planes())

    if column != 1:
        undup=True
    elif column == 1 and num_planes > 1:
        print(f"Skipping session {sess_id}")
        continue # really just don't want to perform decoding on column 1, all duplicated
    elif column == 1 and num_planes == 1:
        undup=False

    # apply multiprocessing to run decoding in parallel
    print(f"Processing session {sess_id}, column {column}, unduplicated: {undup}")
    Parallel(n_jobs=-1)(delayed(run_decoding)(
        session=sess,
        plane=plane,
        stimulus_type=stim_type,
        repetitions=1000,
        decode_dim=dim,
        max_neighbors=15,
        metric="correlation",
        # test_size=0.2,
        bootstrap=True,
        bootstrap_size=30,
        metrics_df=metrics_df,
        unduplicated=undup,
        folder_name="/home/naomi/Desktop/data",
        save_decoding=True,
        results_folder="/home/naomi/Desktop/data/decoding_results",
        tag="2025_0724_3"
    ) for plane in sess.get_planes() for stim_type in stim_types for dim in decode_dims[stim_type])

    
    # for plane in sess.get_planes():
    #     for stim_type in stim_types:
    #         decoding = decode_dims[stim_type]
    #         for dim in decoding:
    #             print(f"Performing decoding for session {sess_id}, plane {plane}, stimulus {stim_type}, dimension {dim}")
    #             run_decoding(
    #                 session=sess,
    #                 plane=plane,
    #                 stimulus_type=stim_type,
    #                 repetitions=1000,
    #                 decode_dim=dim,
    #                 max_neighbors=15,
    #                 metric="correlation",
    #                 # test_size=0.2,
    #                 bootstrap=True,
    #                 bootstrap_size=30,
    #                 metrics_df=metrics_df,
    #                 unduplicated=undup,
    #                 folder_name="/home/naomi/Desktop/data",
    #                 save_decoding=True,
    #                 results_folder="/home/naomi/Desktop/data/decoding_results",
    #                 tag="2025_0724_2"
    #             )
        # print(f"Performing decoding for session {sess_id}, plane {plane}")
        # run_decoding(
        #     session=sess,
        #     plane=plane,
        #     stimulus_type="drifting_gratings_full",
        #     repetitions=2,
        #     decode_dim="direction",
        #     max_neighbors=15,
        #     metric="correlation",
        #     # test_size=0.2,
        #     bootstrap=True,
        #     bootstrap_size=30,
        #     metrics_df=metrics_df,
        #     unduplicated=undup,
        #     folder_name="/home/naomi/Desktop/data",
        #     save_decoding=True,
        #     results_folder="/home/naomi/Desktop/data/decoding_results",
        #     tag="2025_0723"
        # )

Skipping session M409828_11
Skipping session M409828_12
Skipping session M409828_13
Skipping session M409828_14
Skipping session M409828_15
Processing session M409828_16, column 1, unduplicated: False
Processing session M409828_17, column 1, unduplicated: False
Processing session M409828_18, column 1, unduplicated: False
Processing session M409828_19, column 1, unduplicated: False
Processing session M409828_1a, column 1, unduplicated: False
Processing session M409828_1b, column 1, unduplicated: False
Processing session M409828_1c, column 1, unduplicated: False
Processing session M409828_1d, column 1, unduplicated: False
Processing session M409828_1e, column 1, unduplicated: False
Processing session M409828_1f, column 1, unduplicated: False
Processing session M409828_21, column 2, unduplicated: True
Not enough X_data, only have 4 rois, cannot perform decoding
Not enough X_data, only have 4 rois, cannot perform decoding
Not enough X_data, only have 4 rois, cannot perform decoding
Not eno

In [11]:
stim_table = sess.get_stimulus_table("drifting_gratings_full")[0]
stim_table

,start,end,temporal_frequency,spatial_frequency,direction
0,49.829754,51.814732,1.0,0.08,330.0
1,52.832314,54.817272,1.0,0.08,180.0
2,55.834743,57.819733,1.0,0.04,120.0
3,58.837273,60.822262,1.0,0.04,60.0
4,61.839733,63.824722,1.0,0.08,180.0
...,...,...,...,...,...
195,3027.306641,3029.291504,1.0,0.08,60.0
196,3030.309082,3032.293945,1.0,0.04,0.0
197,3033.311523,3035.296631,NaN,NaN,NaN
198,3036.313965,3038.299072,1.0,0.08,180.0


In [12]:
stim_table[stim_table['end'] < sess.get_traces(plane=1, trace_type="dff").time.values[-1]]

,start,end,temporal_frequency,spatial_frequency,direction
0,49.829754,51.814732,1.0,0.08,330.0
1,52.832314,54.817272,1.0,0.08,180.0
2,55.834743,57.819733,1.0,0.04,120.0
3,58.837273,60.822262,1.0,0.04,60.0
4,61.839733,63.824722,1.0,0.08,180.0
...,...,...,...,...,...
195,3027.306641,3029.291504,1.0,0.08,60.0
196,3030.309082,3032.293945,1.0,0.04,0.0
197,3033.311523,3035.296631,NaN,NaN,NaN
198,3036.313965,3038.299072,1.0,0.08,180.0


In [13]:
select_rois(
    session=sess,
    plane=1,
    metrics_df=metrics_df,
    unduplicated=True
)

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 37,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 50,
 52,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 63,
 64,
 67,
 68,
 71,
 72,
 73,
 74,
 75,
 79,
 80,
 81,
 82,
 86,
 88,
 92,
 94,
 97,
 98,
 99,
 101,
 103,
 104,
 106,
 107,
 113,
 116,
 117,
 118,
 119,
 121,
 122,
 124,
 125,
 126,
 129,
 131,
 133,
 138,
 140,
 142,
 146,
 147,
 151,
 153,
 156,
 162,
 166,
 167,
 172,
 177,
 182,
 184,
 191,
 192,
 193,
 198,
 201,
 202,
 204,
 207,
 212,
 214,
 217,
 223,
 226,
 227,
 230,
 233,
 244,
 261,
 263,
 264,
 266,
 270,
 271,
 274,
 276,
 280,
 284,
 286,
 291,
 292,
 295,
 296,
 310,
 319,
 327,
 357,
 359]

In [14]:
get_mean_dff_traces(
    session=sess,
    stimulus_type="drifting_gratings_full",
    plane=1,
    folder_name="/home/naomi/Desktop/data",
)

,0,1,2,3,4,5,6,7,8,9,...,291,292,295,296,309,310,319,327,357,359
0,0.076630,0.050198,0.021158,0.012369,-0.003766,0.061699,0.022833,0.105622,0.352126,0.068923,...,0.137071,0.013756,0.010966,0.000800,0.114272,0.008165,0.011427,0.019785,0.007083,0.031095
1,0.191171,0.037819,0.118133,-0.020995,0.017456,0.052479,0.018865,0.748987,0.412527,-0.018973,...,0.102111,0.027250,0.044454,0.004866,0.067998,0.090045,-0.012796,0.039855,-0.004784,-0.004492
2,0.063505,0.031662,0.354846,-0.011883,0.006509,-0.022964,0.346249,0.286133,0.643897,0.015560,...,0.172292,-0.019094,0.010610,0.007445,0.017504,0.017487,-0.027230,0.042113,0.010675,0.041369
3,0.000696,0.778801,0.194309,0.011102,-0.002468,0.019410,0.202831,-0.017392,0.393902,0.118796,...,0.047024,-0.012273,0.008494,-0.005329,-0.001296,0.005651,0.007675,0.005387,-0.002402,-0.025627
4,0.100667,0.446492,0.117834,0.008556,0.026115,0.012539,0.018741,0.041144,0.254375,0.187516,...,0.009480,0.033650,0.024660,0.041377,-0.050061,0.021851,-0.024749,-0.017389,-0.017017,0.009578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.019533,0.003874,0.128672,-0.006841,0.077706,0.107034,0.029987,0.175765,-0.003198,0.001470,...,0.012608,0.041260,0.023199,0.023562,0.047777,-0.004849,-0.000205,0.011631,0.009306,0.018759
188,0.002852,0.329064,0.024919,0.005679,-0.047621,0.127934,0.005141,0.000168,0.003268,0.004664,...,-0.021486,-0.004284,0.066834,-0.018528,0.001766,0.001833,0.018933,0.012359,-0.005730,0.024559
189,0.042178,0.239574,0.024948,0.023494,-0.073786,0.001421,0.012662,0.036683,0.008961,-0.002515,...,0.009809,-0.013383,-0.017028,-0.015654,-0.018168,-0.028009,0.022127,0.016437,-0.004702,0.003011
190,-0.014846,0.001493,0.044866,0.015025,0.104013,0.019827,-0.003972,0.070244,0.024397,0.106891,...,0.048970,0.006796,0.014113,-0.005009,-0.046434,-0.035654,0.028125,0.004191,0.009026,0.029781


In [15]:
session_id = "M"

In [16]:
get_X_data(session=sess, planes=[1], stimulus_type="drifting_gratings_full", metrics_df=metrics_df, unduplicated=True)

TypeError: get_X_data() got an unexpected keyword argument 'planes'